# FloPy

### Demo of netCDF and shapefile export capabilities within the flopy export module. 

In [1]:
from __future__ import print_function
import os
import flopy

Load our old friend...the Freyberg model

In [2]:
nam_file = "freyberg.nam"
model_ws = os.path.join("data")
ml = flopy.modflow.Modflow.load(nam_file,model_ws=model_ws,verbose=True)
print(ml.dis.sr)


Creating new model with name: freyberg
--------------------------------------------------

Parsing the namefile --> data/freyberg.nam
Setting filehandles:

--------------------------------------------------
External unit dictionary:
{19: filename:data/freyberg.rch, filetype:RCH, 2: filename:data/freyberg.list, filetype:LIST, 11: filename:data/freyberg.dis, filetype:DIS, 13: filename:data/freyberg.bas, filetype:BAS6, 14: filename:data/freyberg.oc, filetype:OC, 15: filename:data/freyberg.lpf, filetype:LPF, 18: filename:data/freyberg.riv, filetype:RIV, 51: filename:data/freyberg.hds, filetype:DATA(BINARY), 20: filename:data/freyberg.wel, filetype:WEL, 53: filename:data/freyberg.cbc, filetype:DATA(BINARY), 52: filename:data/freyberg.ddn, filetype:DATA(BINARY), 27: filename:data/freyberg.pcg, filetype:PCG}
--------------------------------------------------

ModflowBas6 free format:True

loading dis package file...
   Loading dis package with:
      1 layers, 40 rows, 20 columns, and 1 stre

### Some netCDF export capabilities:

#### Export a 2d array

In [3]:
# export a 2d array
ml.dis.top.export(os.path.join(model_ws, 'top.nc'))
ml.dis.top.export(os.path.join(model_ws, 'top.shp'))

2016-06-27 16:29:50.671378 WARNING: removing existing nc file: data/top.nc

2016-06-27 16:29:50.672511 WARNING: start datetime:1970-01-01T00:00:00Z

2016-06-27 16:29:50.672540 starting: initializing attributes

2016-06-27 16:29:50.672676 finished: initializing attributes, took: 0:00:00.000136

2016-06-27 16:29:50.672697 starting: initializing file

2016-06-27 16:29:50.672725 starting: initializing geometry

2016-06-27 16:29:50.674280 starting: building grid crs using proj4 string: +init=EPSG:4326

2016-06-27 16:29:50.674417 finished: building grid crs using proj4 string: +init=EPSG:4326, took: 0:00:00.000137

2016-06-27 16:29:50.674701 starting: projecting grid cell center arrays from +units=m +init=epsg:4326 to +units=m +init=epsg:4326 

2016-06-27 16:29:50.675098 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x1117b45e8> to <pyproj.Proj object at 0x1117b4648>

2016-06-27 16:29:50.675297 finished: initializing geometry, took: 0:00:00.002572

2016-06-27 16:29

#### Export a 3d array

In [4]:
#export a 3d array
ml.lpf.hk.export(os.path.join(model_ws, 'hk.nc'))
ml.lpf.hk.export(os.path.join(model_ws, 'hk.shp'))

2016-06-27 16:29:50.807230 WARNING: removing existing nc file: data/hk.nc

2016-06-27 16:29:50.808304 WARNING: start datetime:1970-01-01T00:00:00Z

2016-06-27 16:29:50.808345 starting: initializing attributes

2016-06-27 16:29:50.808550 finished: initializing attributes, took: 0:00:00.000205

2016-06-27 16:29:50.808571 starting: initializing file

2016-06-27 16:29:50.808600 starting: initializing geometry

2016-06-27 16:29:50.808632 starting: building grid crs using proj4 string: +init=EPSG:4326

2016-06-27 16:29:50.808672 finished: building grid crs using proj4 string: +init=EPSG:4326, took: 0:00:00.000040

2016-06-27 16:29:50.808789 starting: projecting grid cell center arrays from +units=m +init=epsg:4326 to +units=m +init=epsg:4326 

2016-06-27 16:29:50.808837 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x114fd5ca8> to <pyproj.Proj object at 0x114fd5348>

2016-06-27 16:29:50.808882 finished: initializing geometry, took: 0:00:00.000282

2016-06-27 16:29:

#### Export a number of things to the same netCDF file

In [5]:
# export lots of things to the same nc file
fnc = ml.dis.botm.export(os.path.join(model_ws, 'test.nc'))
ml.lpf.hk.export(fnc)
ml.dis.top.export(fnc)

# export transient 2d
ml.rch.rech.export(fnc)


2016-06-27 16:29:50.868864 WARNING: removing existing nc file: data/test.nc

2016-06-27 16:29:50.869701 WARNING: start datetime:1970-01-01T00:00:00Z

2016-06-27 16:29:50.869741 starting: initializing attributes

2016-06-27 16:29:50.870039 finished: initializing attributes, took: 0:00:00.000298

2016-06-27 16:29:50.870089 starting: initializing file

2016-06-27 16:29:50.870128 starting: initializing geometry

2016-06-27 16:29:50.870165 starting: building grid crs using proj4 string: +init=EPSG:4326

2016-06-27 16:29:50.870205 finished: building grid crs using proj4 string: +init=EPSG:4326, took: 0:00:00.000040

2016-06-27 16:29:50.870321 starting: projecting grid cell center arrays from +units=m +init=epsg:4326 to +units=m +init=epsg:4326 

2016-06-27 16:29:50.870371 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x114fd5d08> to <pyproj.Proj object at 0x114fd5348>

2016-06-27 16:29:50.870416 finished: initializing geometry, took: 0:00:00.000288

2016-06-27 16:2

### Export whole packages to a netCDF file

In [6]:
# export mflist
fnc = ml.wel.export(os.path.join(model_ws, 'packages.nc'))
ml.lpf.export(fnc)
fnc.nc

2016-06-27 16:29:50.894565 WARNING: removing existing nc file: data/packages.nc

2016-06-27 16:29:50.895520 WARNING: start datetime:1970-01-01T00:00:00Z

2016-06-27 16:29:50.895545 starting: initializing attributes

2016-06-27 16:29:50.895666 finished: initializing attributes, took: 0:00:00.000121

2016-06-27 16:29:50.895687 starting: initializing file

2016-06-27 16:29:50.895717 starting: initializing geometry

2016-06-27 16:29:50.895754 starting: building grid crs using proj4 string: +init=EPSG:4326

2016-06-27 16:29:50.895802 finished: building grid crs using proj4 string: +init=EPSG:4326, took: 0:00:00.000048

2016-06-27 16:29:50.895970 starting: projecting grid cell center arrays from +units=m +init=epsg:4326 to +units=m +init=epsg:4326 

2016-06-27 16:29:50.896040 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x114fd5348> to <pyproj.Proj object at 0x114fd5ee8>

2016-06-27 16:29:50.896093 finished: initializing geometry, took: 0:00:00.000376

2016-06-27 

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format UNDEFINED):
    Conventions: CF-1.6
    date_created: 2016-06-27T06:29:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -27.3325004578
    geospatial_vertical_max: -17.3986301422
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 0.0
    origin_y: 10000.0
    origin_crs: +init=EPSG:4326
    grid_rotation_from_origin: 0.0
    namefile: freyberg.nam
    solver_flux_tolerance: 0.001
    model_ws: data
    flopy_sr_xul: 0.0
    modflow_version: mf2005
    flopy_sr_yul: 10000.0
    create_hostname: IGSAAAHMLTW272M.local
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/examples/Notebooks
    start_datetime: 1/1/1970
    solver_head_tolerance: 1e-05
    flopy_sr_proj4_str: +init=EPSG:4326
    create_platform: Darwin
    exe_name: mf2005.exe
    flopy_sr_rotation: 0.0
    dimensions(sizes): time(1), layer(1), y(40), x(20)
    variables(dimension

### Export the whole model to a netCDF and a shapefile

In [7]:
fnc = ml.export(os.path.join(model_ws, 'model.nc'))
ml.export(os.path.join(model_ws, 'model.shp'))
fnc.nc

2016-06-27 16:29:50.916410 WARNING: removing existing nc file: data/model.nc

2016-06-27 16:29:50.917103 WARNING: start datetime:1970-01-01T00:00:00Z

2016-06-27 16:29:50.917125 starting: initializing attributes

2016-06-27 16:29:50.917241 finished: initializing attributes, took: 0:00:00.000116

2016-06-27 16:29:50.917256 starting: initializing file

2016-06-27 16:29:50.917278 starting: initializing geometry

2016-06-27 16:29:50.917307 starting: building grid crs using proj4 string: +init=EPSG:4326

2016-06-27 16:29:50.917341 finished: building grid crs using proj4 string: +init=EPSG:4326, took: 0:00:00.000034

2016-06-27 16:29:50.917494 starting: projecting grid cell center arrays from +units=m +init=epsg:4326 to +units=m +init=epsg:4326 

2016-06-27 16:29:50.917562 starting: projecting grid cell center arrays from <pyproj.Proj object at 0x114fd5f48> to <pyproj.Proj object at 0x114fd5ee8>

2016-06-27 16:29:50.917613 finished: initializing geometry, took: 0:00:00.000335

2016-06-27 16:

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format UNDEFINED):
    Conventions: CF-1.6
    date_created: 2016-06-27T06:29:00Z
    geospatial_vertical_positive: up
    geospatial_vertical_min: -27.3325004578
    geospatial_vertical_max: -17.3986301422
    geospatial_vertical_resolution: variable
    featureType: Grid
    origin_x: 0.0
    origin_y: 10000.0
    origin_crs: +init=EPSG:4326
    grid_rotation_from_origin: 0.0
    namefile: freyberg.nam
    solver_flux_tolerance: 0.001
    model_ws: data
    flopy_sr_xul: 0.0
    modflow_version: mf2005
    flopy_sr_yul: 10000.0
    create_hostname: IGSAAAHMLTW272M.local
    create_directory: /Users/jdhughes/Documents/Development/flopy_git/examples/Notebooks
    start_datetime: 1/1/1970
    solver_head_tolerance: 1e-05
    flopy_sr_proj4_str: +init=EPSG:4326
    create_platform: Darwin
    exe_name: mf2005.exe
    flopy_sr_rotation: 0.0
    dimensions(sizes): time(1), layer(1), y(40), x(20)
    variables(dimension